In [17]:
!pip3 install langchain langchain-openai
!pip3 install pytz


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 75 kB 6.6 MB/s eta 0:00:011
     |████████████████████████████████| 997 kB 22.9 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 63.3 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
import importlib.util
print(importlib.util.find_spec("langchain.chat_models") is not None)

True


In [39]:
# 환경 설정
from typing import List, Dict, Any, Optional
from getpass import getpass

import sys, os

# src 폴더 경로를 동적으로 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

try:
    from langchain_openai import ChatOpenAI
    from langchain.prompts import ChatPromptTemplate
    from langchain.chains import LLMChain
    LANGCHAIN_AVAILABLE = True
except ImportError:
    LANGCHAIN_AVAILABLE = False

from src.config import get_config
from src.utils.logging_config import get_logger

print("----------")
print(f"LangChain available: {LANGCHAIN_AVAILABLE}")

----------
LangChain available: True


In [40]:
class LLMProcessor:
    def __init__(
        self,
        model: Optional[str] = None,
        temperature: Optional[float] = None,
        config=None,
        api_key: Optional[str] = None
    ):
        # Config & Logger
        self.config = config or get_config()
        self.logger = get_logger(__name__)

        self.model_name = model or self.config.OPENAI_MODEL
        self.temperature = temperature if temperature is not None else self.config.OPENAI_TEMPERATURE

        # API 키 입력 처리
        if api_key is None or not api_key.strip():
            api_key_input = getpass("🔑 OpenAI API Key를 입력하세요: ").strip()
            api_key = api_key_input if api_key_input else None

        if not api_key:
            raise ValueError("OpenAI API Key가 제공되지 않았습니다.")

        # 환경 변수에 등록 (LangChain에서 자동 사용)
        os.environ["OPENAI_API_KEY"] = api_key

        # LangChain 초기화
        if LANGCHAIN_AVAILABLE:
            self.llm = ChatOpenAI(model=self.model_name, temperature=self.temperature)
            self.logger.info(f"LLMProcessor 초기화 완료 (model={self.model_name}, temperature={self.temperature})")
        else:
            self.logger.error("LangChain이 설치되지 않았습니다.")
            raise ImportError("LangChain이 설치되지 않았습니다.")

    def generate_response(self, query: str, retrieved_chunks: List[Dict[str, Any]]) -> str:
        """검색된 청크를 바탕으로 LLM 응답 생성"""
        self.logger.info(f"LLM 응답 생성 시작: query='{query[:50]}...'")

        # 컨텍스트 구성
        if not retrieved_chunks:
            context = self.config.NO_CONTEXT_MESSAGE
            self.logger.warning("검색된 청크가 없습니다.")
        else:
            context_parts = [
                self.config.CONTEXT_FORMAT.format(
                    index=i+1,
                    file_name=chunk.get("file_name", "unknown"),
                    chunk_text=chunk.get("chunk_text", "")
                )
                for i, chunk in enumerate(retrieved_chunks)
            ]
            context = "\n\n".join(context_parts)

        # 프롬프트 생성
        prompt = ChatPromptTemplate.from_template(self.config.RAG_PROMPT_TEMPLATE)
        chain = prompt | self.llm

        try:
            response = chain.invoke({"context": context, "query": query})
            self.logger.info(f"LLM 응답 생성 완료: {len(response.content)} 문자")
            return response.content
        except Exception as e:
            self.logger.error(f"응답 생성 중 오류 발생: {e}")
            return f"응답 생성 중 오류 발생: {e}"

In [41]:
def run_llm_demo() -> None:
    """
    OpenAI ChatOpenAI를 사용하여 LLMProcessor 동작을 확인하는 최소 실행 예제
    """
    # 더미 검색 청크
    retrieved_chunks = [
        {
            "file_name": "policy_overview.txt",
            "chunk_text": "이 문서는 회사의 내부 정책(출퇴근, 휴가 등)에 대해 요약합니다. 주요 항목은 유연근무제, 연차 규정 등입니다."
        },
        {
            "file_name": "project_plan.md",
            "chunk_text": "프로젝트 일정: 기획(2주), 개발(8주), 테스트(2주). 주요 마일스톤은 요구사항 확정과 첫 배포입니다."
        }
    ]

    query = "회사 정책과 프로젝트 일정을 간단히 요약해줘."

    try:
        processor = LLMProcessor()  # ✅ 실행 시 API Key 입력 프롬프트
        response = processor.generate_response(query=query, retrieved_chunks=retrieved_chunks)
        print("\n=== LLM 응답 시작 ===\n")
        print(response)
        print("\n=== LLM 응답 끝 ===\n")
    except Exception as e:
        print(f"데모 실행 중 오류 발생: {e}")


# 데모 실행
run_llm_demo()

2025-11-11 17:40:36 [I] __main__ - LLMProcessor 초기화 완료 (model=gpt-4o-mini, temperature=0.0)
2025-11-11 17:40:36 [I] __main__ - LLM 응답 생성 시작: query='회사 정책과 프로젝트 일정을 간단히 요약해줘....'
2025-11-11 17:40:38 [I] __main__ - LLM 응답 생성 완료: 117 문자

=== LLM 응답 시작 ===

회사의 내부 정책은 유연근무제와 연차 규정 등을 포함하고 있습니다. 프로젝트 일정은 기획 단계가 2주, 개발 단계가 8주, 테스트 단계가 2주로 구성되어 있으며, 주요 마일스톤은 요구사항 확정과 첫 배포입니다.

=== LLM 응답 끝 ===

